Coursera의 Neural Networks for Machine Learning 강의를 정리한 내용임.   
2016.12.27. by Dongwan Kim

# Week12. Restricted Boltzmann machines(RBMs)

### Lecture 12a, Boltzmann machine learning

<u>The goal of learning</u>

week 11의 말미에 Stochastic Hopfield nets with hidden units이 바로 Boltzmann machines이라는 것과, 이것을 이용해 binary data vector $\mathbf{v}$에 대한 확률 분포 $p(\mathbf{v})$를 구하는 방법에 대해 알아보았다.

$$p(\mathbf{v}) = \frac{\sum_h e^{-E(\mathbf{v}, \mathbf{h})}}{\sum_{\mathbf{u}, \mathbf{g}} e^{-E(\mathbf{u}, \mathbf{g})}}$$

이때 $n$개의 binary data vector($v_i$)로 구성된 training data을 Boltzmann machine에 학습시킨다는 것은 학습시키고자 하는 Boltzmann machines이 이 $n$개의 binary data vector에 할당하는 확률의 곱(결합 확률)이 최대가 되도록하는 machine의 weight vector를 찾는 것이다.(확률 곱 혹은 log 확률의 합을 최대로 하는 것)

$$\arg\max_{\mathbf{w}}\left( \prod_{i}^{n} p(\mathbf{v}_i)\right) \\ \mathrm{or}\\  \arg\max_{\mathbf{w}}\left( \sum_{i}^{n} \log p(\mathbf{v}_i)\right)$$ 

혹은 아래와 같은 과정으로 $n$개의 binary vector를 sampling했을 때 training data와 같은 binary vector들을 얻을 확률이 최대가 되도록 machine의 weight vector를 찾는 것이라고 해석할 수도 있다.

1. random하게 visible layer와 hidden layer의 unit들의 state에 binary 값을 할당    
1-1. random하게 unit을 선택하고 energy gap을 구하여 energy가 감소 되도록 state값을 update   
1-2. thermal equilibrium에 이를 때까지 1-1과정을 반복
1-3. thermal equilibrium에 이르면 visible vector하나를 sampling($v_i)$
2. 1과정을 $n$번 반복

<u>Why the learning could be difficult</u>

weight들을 학습시키는 과정은 생각보다 단순하지 않을 수 있다. 예를들어 아래와 같은 RBM이 주어졌고, training data는 (1, 0)과 (0, 1)이라 할 때 어떤 weight값들을 사용해야 할까? 즉 두개의 visible unit의 state값이 반대인 상황인데, 이런 데이터 조합이 나올 확률이 최대화하려면 5개의 weight의 곱이 음수가 되어야 한다. 즉 어떤 하나의 weight값을 결정하기 위해서는 다른 모든 weight값을 고려해야 한다.

<img  src="./_images/12_simple_bm.PNG"/>

<u>A very surprising fact</u>

그런데 매우 놀랍게도...      
$\frac{\partial ~ \mathrm{log} ~ p(\mathbf{v})}{\partial ~ w_{ij}}$가 아래와 같이 유도 된다. (유도 과정은 <a href="http://image.diku.dk/igel/paper/AItRBM-proof.pdf">이곳</a>의 4.1을 참고하기 바람)     
즉 $w_{ij}$을 조절하기 위한 gradient정보는       
visible layer가 주어졌을 때(Positive phase)와 그렇지 않을 때(Negative phase)      
인접한 unit의 state값으로 충분하다는 것이다.

$$\frac{\partial ~ \mathrm{log} ~ p(\mathbf{v})}{\partial ~ w_{ij}} = \left< s_i s_j\right>_{\mathbf{v}} - \left< s_i s_j\right>_{model}$$

위 식에서 $\left< s_i s_j\right>_{\mathbf{v}}$는 visible layer를 training data의 binary vector로 고정한 후  thermal equilibrium에서 $w_{ij}$로 연결되는 두 unit 값을 sampling했을 때 두 state값이 모두 1인 빈도, 그리고 $\left< s_i s_j\right>_{model}$는 모든 state값을 random값으로 시작했을 때 thermal equilibrium에서 $w_{ij}$로 연결되는 두 unit값을 sampling했을 때 이 값이 모두 1인 빈도라고 생각할 수 있다.

또한 $\left< s_i s_j\right>_{\mathbf{v}}$에 의하면 visible layer를 고정 했을 때 $w_{ij}$로 연결되는 두 unit값이 모두 1인 빈도가 크다면 $w_{ij}$를 보다 크게 해야한다는 것이다.(Hebbian learning rule의 간단한 형태임).
반면 $\left< s_i s_j\right>_{model}$는 weight들이 지나치게 커지는 것을 막아주는데, spurious minima를 제거하는 과정(unlearning)과 같은 역할을 하게 된다.

<u>Why is the derivative so simple?</u>

week11에서 다뤘다시피 probability of a configuration of the visible units는 아래와 같이 $-E$에 대한 exponential function 이다.

$$p(\mathbf{v}) = \frac{\sum_h e^{-E(\mathbf{v}, \mathbf{h})}}{\sum_{\mathbf{u}, \mathbf{g}} e^{-E(\mathbf{u}, \mathbf{g})}}$$

따라서 log probability는 아래와 같이 $-E$에 대한 linear function이다.

$$\mathrm{log} ~ p(\mathbf{v}) = \sum_{h} \left[ -E(\mathbf{v}, \mathbf{h}) \right] - \sum_{\mathbf{u}, \mathbf{g}} \left[ -E(\mathbf{u}, \mathbf{g}) \right]$$

그런데 energy of a joint configuration은 아래와 같이 $w$에 대한 linear function이므로

$$-E(\mathbf{v}, \mathbf{h}) = \sum_{i \in {vis}} v_i b_i + \sum_{k \in {hid}} h_k b_k + \sum_{i < j} v_i v_j w_{ij} + \sum_{i, k} v_i h_k w_{ik}+ \sum_{k < l} h_k h_l w_{kl}$$

$$-\frac{\partial E}{\partial w_{ij}} = s_i s_j$$

$-\frac{\partial E}{\partial w_{ij}}$는 위와 같고, log probability 또한 $w$에 대한 linear function이다.

<u>Why do we need the negative phase?</u>

learning은 visible layer를 고정시키고 진행되는 [positive phase]와 고정된 layer없이 진행되는 [negative phase]로 나뉜다. 두 phase 모두 thermal equilibrium에 이르도록 state update을 진행하는데 결국 $p(\mathbf{v})$를 최대화 하는 것을 목표로 한다.

아래 $p(\mathbf{v})$의 분자가 결국 weight gradient $\frac{\partial ~ \mathrm{log} ~ p(\mathbf{v})}{\partial ~ w_{ij}}$의 첫번째 항인 $\left< s_i s_j\right>_{\mathbf{v}}$의 역할을 하고, 분모가 두번째 항인 $\left< s_i s_j\right>_{model}$가 되는데, $p(\mathbf{v})$를 최대화하기 위해서는 (positive phase에서) visible layer가 고정되었을 때 thermal equilibrium이 되도록 하는 $\mathbf{h}$를 찾고 $E(\mathbf{v}, \mathbf{h})$를 최소화 하는 $\mathbf{w}$를 찾아야 한다. 또한 (negative phase에서) thermal equilibrium이 되도록 하는 global configuration을 찾고 $E(\mathbf{u}, \mathbf{g})$를 최대화 하는 $\mathbf{w}$를 찾아야 한다.

$$p(\mathbf{v}) = \frac{\sum_{h} e^{-E(\mathbf{v}, \mathbf{h})}}{\sum_{\mathbf{u}, \mathbf{g}} e^{-E(\mathbf{u}, \mathbf{g})}}$$

따라서 

negative phase는 unlearning(Hopfield net에서 spurious minima를 제거하기 위해 거치는 과정)과 같다고 볼 수 있다.

$$\frac{\partial ~ \mathrm{log} ~ p(\mathbf{v})}{\partial ~ w_{ij}} = \left< s_i s_j\right>_{\mathbf{v}} - \left< s_i s_j\right>_{model}$$

<u>An inefficient way to collect the statistics required for learning</u>

learning을 위해서는 $\left< s_i s_j\right>_{\mathbf{v}}$와 $\left< s_i s_j\right>_{model}$를 구해야 한다. 첫번째 것은 positive phase에서 구할 수 있는 statistics이고, 두번째 것은 negative phase에서 구할 수 있는 statistics이다.(Hinton and Sejnowski, 1983)

> **Positive phase**
> 1. visible unit들은 data vector로 고정시키고, hidden unit들을 random binary state으로 설정
> 2. network가 thermal equilibrium(at temperature of 1)에 이르도록 update을 진행(random sequential update)
> 3. 각 unit pair에 대해 $\left< s_i s_j\right>$를 sampling (각 pair마다 얼마나 자주 paired unit 둘다 on인지 구함)
> - 모든 training set내 모든 data vector에 대해 위 1, 2, 3 과정을 반복하고, 구해진 $\left< s_i s_j\right>$들의 평균($\left< s_i s_j\right>_{\mathbf{v}}$)을 구함

> **Negative phase**
> 1. 모든 unit(visible + hidden)들을 random binary state으로 설정
> 2. network가 thermal equilibrium(at temperature of 1)에 이르도록 update을 진행(random sequential update)
> 3. 각 unit pair에 대해 $\left< s_i s_j\right>$를 sampling (각 pair마다 얼마나 자주 paired unit 둘다 on인지 구함)
> 4. 1, 2, 3 과정을 여러번(얼마나 자주?) 반복하고, 구해진 $\left< s_i s_j\right>$들의 평균($\left< s_i s_j\right>_{model}$)을 구함 

> (이후 converge할 때까지 weight change와 Positive phase, Negative phase의 반복?.. 이 때문에 매번 random state에서부터 equilibrium을 찾는 과정이 반복됨)

### Lecture 12b, More efficient ways to get the statistics. (ADVANCED)

<u>A better way of collecting the statistics</u>

매번 random state에서부터 시작할 경우 thermal equilibrium에 이르기까지 시간이 오래 걸리고, 과연 thermal equilibrium에 도달 했는지 아니면 더 update을 해야 thermal equilibrium에 도달할지 알기 어렵다.

따라서 매번 새로운 random state에서 시작하는 것이 아니라 이전에 thermal equilibrium에 이르렀을 때의 state에서 시작하면 더 효과적이지 않을까?

이전에 저장된 state을 **particle**이라 부르고, particle에서 다시 update을 진행하는 것을 warm start라 한다. 이런 방식은 단지 몇번의 update으로 (weight change 이후) 다시 thermal equilibrium에 이르게 하는 장점이 있다.

<u>Neal's method for collecting the statistics (Radford Neal 1992)</u>

> **Positive phase**
> 1. 각 traning case마다 particle(바로 전 weight change할 때의 hidden layer state)을 보관, 즉 매번 hidden layer를 random으로 시작하는 것이 아님
> 2. 나머지 과정은 (Hinton and Sejnowski, 1983)와 같음

> **Negative phase**
> 1. Negative phase를 위한 수개의 particle을 보관(여기에 사용되는 particle을 **fantasy particle**이라 함), 모든 hidden layer unit을 random으로 시작하는 것이 아니라 fantasy particle에서 시작함.
> 2. 나머지 과정은 (Hinton and Sejnowski, 1983)와 같음

<u>Adapting Neal's approach to handle mini-batches</u>

.

<u>The simple mean field approximation</u>

.

<u>An efficient mini-batch learning procedure for Boltzmann Machines (Salakhutdinov & Hinton 2012)</u>

.

<u>Making the updates more parallel</u>

.

<u>Making the updates more parallel</u>

.

<u>Can a DBM learn a good model of the MINIST digits?</u>

.

<u>A puzzle</u>

.

<u>The learning raises the effective mixing rate</u>

.

<u>How fantasy particles move between the model's modes</u>

.

### Lecture 12c, Restricted Boltzmann Machines

<u>Restricted Boltzmann Machines</u>

Hidden unit간 connection이 없는 형태의 Boltzmann Machine을 **Restricted Boltzmann Machines**라 한다. Boltzmann Machine에 이런 제약조건이 추가되면 (if the visible units are given) equilibrium distribution of the hidden units를 바로 구할 수 있다. 

아래 그림과 같은 형태인데 hidden layer가 하나 이고 hidden layer unit간 연결이 없고, visible layer unit간 연결도 없다. Computer scientists들은 이런 형태의 graph를 'bipartite graph'라 한다.

<img  src="./_images/12_rbm1.PNG"/>

앞서 Boltzmann Machine에서 learning을 위한 $\Delta w_{ij}$를 아래와 같이 구할 수 있었고, $\left< s_i s_j\right>_{\mathbf{v}}$를 구하기 위해 visible layer를 data vector로 고정하고 equilibrium distribution을 찾아야 했었다. 

$$\Delta w_{ij} \propto \frac{\partial ~ \mathrm{log} ~ p(\mathbf{v})}{\partial ~ w_{ij}} = \left< s_i s_j\right>_{\mathbf{v}} - \left< s_i s_j\right>_{model}$$

그런데 RBM에서는 layer내 unit들이 서로 독립이기 때문에 unit $i$의 state이 1일 확률 $p(h_i = 1)$과 unit $j$의 state이 1일 확률 $p(h_j = 1)$을 바로 구할 수 있고, 이 때문에 $\left< s_i s_j\right>_{\mathbf{v}}$ 혹은 $\left< v_i h_j\right>_{\mathbf{v}}$를 쉽게 구할 수 있다. 

$$p(h_j = 1) = \frac{1}{1 + e^{-(b_j + \sum_{i \in {vis}} v_i w_{ij})}} $$

**Learning algorithms for RBM**

RBM learning을 위한 algorithm이 여러가지 있는데, 여기에서는 PCD(Persistent Constrastive Divergence)와 CD(Constrastive Divergence)를 살펴보겠다.

<u>1) PCD: An efficients mini-batch learning procedure for Restricted Boltzmann Machines (Tieleman, 2008)</u>

2008년 Tileman이 'RBM을 mini-batch 방식으로 학습시킬 때 효과적인 알고리즘'으로 소개한 것으로 Persistent Constrastive Divergence(PCD)로 불린다.(<a href='http://www.cs.utoronto.ca/~tijmen/pcd/pcd.pdf'>Tieleman, 2008</a>)

이 알고리즘에서는..

> **Positive phase**: 
> - visible unit들을 data vector로 고정시키고, 모든 쌍의 $\left< v_i h_j\right>$를 계산
> - mini-batch 내의 모든 data vector에 대하여 $\left< v_i h_j\right>$들을 구한 후 각 pair마다 평균을 구함

> **Negative phase**:
> - fantasy particle을 update(alternating parallel update)
> - 모든 fantasy particle에 대한 $v_i h_j$의 평균을 구함

<u>※ A picture of an inefficient version of the Boltzmann machine learning algorithm for an RBM</u>

앞서 Lecture 12a에서 Boltzmann machine learning을 위한 아이디어로 'An inefficient way to collect the statistics required for learning'를 제시 했었다. 이 아이디어를 RBM에 적용할 경우 아래와 같은 그림을 그려볼 수 있다.

<img  src="./_images/12_rbm_learning1.PNG"/>

각 update 상황을 시점($t$)으로 나눠 볼 것인데,    
- $t=0$: visible layer를 data vector로 설정, hidden layer를 update, 이때 unit $i$와 unit $j$의 correlation을 $\left< v_i h_j\right>^0$이라 함
- $t=1$: $t=0$에서의 hidden layer가 주어졌을 때, visible layer를 update, 다시 hidden layer를 update, 이때 unit $i$와 unit $j$의 correlation을 $\left< v_i h_j\right>^1$이라 함
- ... (update을 계속)
- thermal equilibrium에 이르렀을 때 unit $i$와 unit $j$의 correlation을 $\left< v_i h_j\right>^{\infty}$를 구함

이렇게 구한 statistics를 이용해 weight gradient를 아래와 같이 구함($\epsilon$은 learning rate)
$$\Delta w_{ij} = \epsilon \left( \left< v_i h_j\right>^0 - \left< v_i h_j\right>^{\infty} \right)$$

그런데 이런 접근 법은 얼마나 오랫동안 update을 해야 thermal equilibrium에 이를 것인지와 thermal equilibrium이전에 update을 멈출 경우 learning이 정상적이지 않을 수 있다는 것이다. (물론 update이 완전히 진행되지 않았다고 해서 learning이 조금도 진전되지 않는 것일지에 관해서는 반론의 여지가 있다.)

<u>2) Contrastive divergence: A very surprising short-cut</u>

(바로 반론해보면..) update을 단 두번만 진행한 상태에서의 statistics를 이용해 weight change를 해도 (이론적 허점이 있지만) 빠르고 잘 동작한다.
$$\Delta w_{ij} = \epsilon \left( \left< v_i h_j\right>^0 - \left< v_i h_j\right>^{1} \right)$$

<img  src="./_images/12_rbm_learning2.PNG"/>

<u>Why does the shortcut work?</u>

이런 shortcut(앞서 소개한 Contrastive Divergence)이 잘 동작하는 이유는 뭘까?

초반 visible layer에 data vector값을 할당하고 Markov chain에 의해 전체 configuration이 변해간다.즉 global configuration에서 data의 영향이 점점 줄어들고 weight들의 영향력이 점점 커지게 된다. 그런데 설정한 weight이 좋은 값을 갖고 있지 않다면 equilibrium까지 가는 것이 큰 의미가 없게 된다. 

따라서 weight을 작게 시작하여 초반에는 Markov chain의 몇 step만을 weight change에 반영하고, 점차 반영하는 step수를 증가시키는 아이디어를 생각해 볼 수 있다.

<u>A picture of contrastive divergence learning</u>

Contrastive divergence기법으로 learning을 진행할 때 energy surface를 생각해 보면, Positive statistics를 구하게 되는 energy point는 아래 그림에서 녹색 화살표이고, negative statistics를 구하게 되는 energy point는 아래 그림에서 빨간색 화살표이다.( equilibrium까지 가지 않았으므로 빨간 화살표가 energy surface의 minimum에 있지 않다.)

이때 weight change를 할 경우 energy surface는 아래 두번째 그림과 같이 바뀐다. 즉 positive statistics에서의 energy가 작아도록 그리고 negative statistics에서의 energy가 커지도록 weight들이 바뀌게 된다.

<img  src="./_images/12_rbm_shortcut.PNG"/>

.

<u>When does the shortcut fail?</u>

그렇다면 어떤 경우에 Constrastive divergence가 실패할까?

현재의 model에 들어맞는 outlier가 있을 경우 low energy hole과 같은 역할을 하게 되고 이는 normalization을 커지게 만든다. 대신 Persistent particle을 사용할 경우 이런 log energy hole을 없애는 효과를 갖게 된다. 

따라서 초반에는 작은 weight으로 시작하여 CD1(1 step of Markov chain)을 사용하고, 점차 weight이 커지면서 CD3, CD5, CD10 등을 사용하는 방법을 생각해볼 수 있음

### Lecture 12d, An example of Contrastive Divergence Learning

<u>How to learn a set of features that are good for reconstructing images of the digit 2</u>

16 x 16의 손글씨 데이터에서 숫자 2를 나타내는 데이터만을 이용해, 이를 학습하는 (혹은 feature learning)하는 과정을 살펴보자. 

우선 visible layer는 16x16개의 unit으로 구성되고, hidden layer에는 50개의 binary unit을 배치한다. 이 둘을 연결하는 weight은 16x16x50개가 존재한다. 즉 각 weight을 50개의 image라고 생각하고 관찰할 수 있다. 

Constrastive Divergence를 위해 단지 한단계의 markov update을 진행하여 visible layer에 대한 reconstruction을 생성한다. 이 상태에서 $\Delta w_{ij}$를 구하고 weight change를 진행하고, 다시 다른 data image를 할당하여 위 과정을 반복한다.

$$\Delta w_{ij} = \epsilon \left( \left< v_i h_j\right>^0 - \left< v_i h_j\right>^{1} \right)$$

<img  src="./_images/12_rbm_mnist1.PNG"/>

(바로 반론해보면..) update을 단 두번만 진행한 상태에서의 statistics를 이용해 weight change를 해도 (이론적 허점이 있지만) 빠르고 잘 동작한다.
$$\Delta w_{ij} = \epsilon \left( \left< v_i h_j\right>^0 - \left< v_i h_j\right>^{1} \right)$$

<u>The weights of the 50 feature detectors</u>

weight chage를 진행하면서 16x16x50개의 weight을 살펴보면(각각 50개의 image라고 생각할 수 있다.) 
아래와 같이 점차 숫자 2와 같이 image가 변해감을 관찰할 수 있다.

<img  src="./_images/12_rbm_mnist2.PNG"/>

<img  src="./_images/12_rbm_mnist3.PNG"/>

<u>The final 50 x 256 weights: Each neuron grabs a different feature</u>

<img  src="./_images/12_rbm_mnist4.PNG"/>

<u>How well can we reconstruct dight images from the binary feature activations?</u>

숫자 2를 학습한 weight들이 있을 때, training 과정에 사용하지 않았던 image를 입력했을 때, reconstructed visible layer는 어떠할까?

아래와 같이 숫자 2를 넣으면 정말 숫자 2와 같은 image를 reconstruct하고
다른 숫자(3)을 넣을 경우 다시 조금 찌그러진 2를 reconstruct함을 관찰할 수 있다.

<img  src="./_images/12_rbm_mnist_vs.PNG"/>

### Lecture 12e, RBMs for collaborative filtering

RMB을 collaborative filtering에 적용해보자. 여기서 collaborative filtering이란 어떤 사용자가 과거에 좋아 했던 다른 상품들에 대한 정보와 다른 사용자들의 상품 선호도에 관한 정보를 이용해, 이 사용자가 특정 상품을 과연 좋아할지를 결정하는 알고리즘이라 할 수 있다.

이를 사용하는 예시로 Netflix competition을 살펴볼텐데, 이때 사용된 데이터는 50만명의 사용자가 18000개의 영화에 대한 1000만개의 평가 정보이다.

<u>Collaborative filtering: The Netflix competition</u>

Netflix competition 데이터는 아래와 같이 각 row의 user가 column의 movie를 몇점으로 평가 했는지에 대한 것이다. 이때 물음표와 같이 user4가 movie3를 몇점으로 평가할지를 예측해야 한다.

<img  src="./_images/12_netflix1.PNG"/>

<u>Lets use a "language model"</u>

오래전에 다뤘던 language model을 이용해 이 문제를 풀수도 있다. 우선 데이터를 user, movie, rating으로 triple형태로 바꾼다.

<img  src="./_images/12_lan_model.PNG"/>

그리고 network를 아래와 같이 구성한다.
- U4: Input layer for user
- U4 feat: Layer for distributed encoding of users
- M3: Input layer for movie
- M3 feat: Layer for distributed encoding of movies
- Scalar product
- rating: output layer

U4 feat과 M3 feat 다음에 hidden layer를 추가하지 않은 것이 특이한데, 이렇게 구성한 결과는 matrix factorization(우측 그림)을 하는 것과 동일하다.

<img  src="./_images/12_mat_fac.PNG"/>

<u>An RBM alternative to matrix factorization</u>

.

<u>How to avoid dealing with all those missing ratings</u>

.

<u>How well does it work? (Salakhutdinov et al. 2007)</u>

RBM의 성능도 matrix factorization만큼 나오는데, error case가 두 방법에서 매우 다르다. 따라서 두 방법을 averaging해서 사용할 경우 매우 효과적일 수 있다. (2007년 Netflix competition에서 우승한 알고리즘은 RBM을 포함한 여러 방법론의 결과를 averaging해서 사용하는 방식이었다.)

.